In [11]:
import torch
import argparse
import time
import math
import os, sys
import itertools
import pickle
from tqdm import tqdm
import numpy as np
import os.path as osp
import numpy as np

In [12]:
torch.cuda.device_count()

1

In [13]:
def loadpkl(path):
    with open(path, 'rb') as f:
        rst=pickle.load(f)
    return rst

In [14]:
def dumppkl(data,path):
    with open(path,'wb') as f:
        pickle.dump(data,f)

In [15]:
def get_pair_and_txt(train_data,full_data):
    cite_pair=[]
    cite2txt_abs_title_author={}
    for k,v in train_data.items():
        data={}
        data['paperID']=k
        if full_data[k]['abstract'] is not None:
            data['abstract']=full_data[k]['abstract']
        else:
            data['abstract']=""
        if full_data[k]['title'] is not None:
            data['title']=full_data[k]['title']
        else:
            data['title']=""
        data['authors']=full_data[k]['authors']
        cite2txt_abs_title_author[k]=data
        for pred in v:
            dp={}
            dp['query']=k
            dp['key']=pred
            dp['value']=get_cover_paper(k,pred,full_data)
            cite_pair.append(dp)

            data={}
            data['paperID']=pred
            if full_data[pred]['abstract'] is not None:
                data['abstract']=full_data[pred]['abstract']
            else:
                data['abstract']=""
            if full_data[pred]['title'] is not None:
                data['title']=full_data[pred]['title']
            else:
                data['title']=""
            data['authors']=full_data[pred]['authors']
            cite2txt_abs_title_author[pred]=data
    return cite_pair,cite2txt_abs_title_author

In [16]:
def get_cover_paper(query,key,full_data):
    query_ref=set([paper['paperId'] for paper in full_data[query]['references']])
    key_ref=set([paper['paperId'] for paper in full_data[key]['references']])
    key_ref.add(key)
    cover_num=len(query_ref&key_ref)
    return cover_num
def calculate_cite_info(train_data,full_data):
    cite_pair=[]
    for k,v in train_data.items():
        for pred in v:
            cite_pair.append(get_cover_paper(k,pred,full_data))
    cite_pair=np.array(cite_pair)
    print(np.sum(cite_pair==0))
    print(len(cite_pair))
    print(np.mean(cite_pair))
    return cite_pair

In [17]:
path="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml/full_data_no_embed.pkl"
full_data=loadpkl(path)

In [18]:
path="/share/data/mei-work/kangrui/github/ssref/result/pretrained_pair_sbert/f1_result/eval_train_pred_1000.pkl"
train_data=loadpkl(path)
path="/share/data/mei-work/kangrui/github/ssref/result/pretrained_pair_sbert/f1_result/eval_dev_pred_1000.pkl"
dev_data=loadpkl(path)

In [19]:
cite_pair,cite2txt_abs_title_author=get_pair_and_txt(train_data,full_data)
path1="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml-small/train_top1000_cite_pair.pkl"
path2="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml-small/train_top1000_cite2txt_abs_title_author.pkl"
dumppkl(cite_pair,path1)
dumppkl(cite2txt_abs_title_author,path2)

In [30]:
len(cite_pair)
rst=[]
for dp in cite_pair:
    rst.append(dp['value'])
print(np.mean(rst))

1.5447476206857012


In [20]:
cite_pair,cite2txt_abs_title_author=get_pair_and_txt(dev_data,full_data)
path1="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml-small/dev_top1000_cite_pair.pkl"
path2="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml-small/dev_top1000_cite2txt_abs_title_author.pkl"
dumppkl(cite_pair,path1)
dumppkl(cite2txt_abs_title_author,path2)

In [28]:
len(cite_pair)
rst=[]
for dp in cite_pair:
    rst.append(dp['value'])
print(np.mean(rst))

2.006386803185438


In [29]:
path1="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml-small/train_top1000_cite_pair.pkl"
path2="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml-small/train_top1000_cite2txt_abs_title_author.pkl"
cite2txt_abs_title_author=loadpkl(path2)
cite_pair=loadpkl(path1)

In [24]:
print(len(cite2txt_abs_title_author))
print(len(cite_pair))

1151773
8721000


In [25]:
cite_pair[5]

{'query': '8ecf5a8459b8a8767a2123294fd6fc1778b2d39c',
 'key': 'eb6eca24a6a5fd913614adb6bcd641aa6f854891',
 'value': 6}

In [26]:
path1="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml-small/dev_top1000_cite_pair.pkl"
path2="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml-small/dev_top1000_cite2txt_abs_title_author.pkl"
cite2txt_abs_title_author=loadpkl(path2)
cite_pair=loadpkl(path1)

In [27]:
cite_pair[5]

{'query': 'cd46a80ce754dfa497e46477f9589e81b9ac9df4',
 'key': '983265d0fe0ba349ec74d6d8e23e7381a9e32e2c',
 'value': 1}

In [9]:
def txt_generate(data):
    para=""
    para+=data['title']+'[SEP]'
    authortxt=' and '.join([item['name'] for item in data['authors']])
    para+=authortxt+'[SEP]'
    para+=data['abstract']
    return para

In [15]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

In [11]:
data=list(cite2txt_abs_title_author.values())[0]

In [20]:
print(txt_generate(data))

Formal Verification of Robustness and Resilience of Learning-Enabled State Estimation Systems for Robotics[SEP]Wei Huang and Yifan Zhou and Youcheng Sun and Alec Banks and Jie Meng and James Sharp and S. Maskell and Xiaowei Huang[SEP]This paper presents a formal verification guided approach for a principled design and implementation of robust and resilient learning-enabled systems. We focus on learning-enabled state estimation systems (LE-SESs), which have been widely used in robotics applications to determine the current state (e.g., location, speed, direction, etc.) of a complex system. The LE-SESs are networked systems composed of a set of connected components including Bayes filters for localisation, and neural networks for processing sensory input. We study LE-SESs from the perspective of formal verification, which determines the satisfiability of a system model against the specified properties. Over LE-SESs, we investigate two key properties - robustness and resilience - and prov

In [28]:
cnt=0
for k in train_data:
    data= cite2txt_abs_title_author[k]
    _inputs = tokenizer.encode_plus(
            txt_generate(data),
            None,
            add_special_tokens=True,
#             max_length=512,
#             padding='max_length',
            return_token_type_ids=True,
            truncation=False,
        )
    if len(_inputs['input_ids'])>512:
        cnt+=1
print(cnt)

Token indices sequence length is longer than the specified maximum sequence length for this model (576 > 512). Running this sequence through the model will result in indexing errors


44
